# Using Selenium, Beatutiful Soup & Requests: _Hershel, O My Bag & Whimsy and Row_

In this section we will be using selenium and beautiful soup to scrape all of the women's bags from Hershel, O My Bag & Whimsy and Row. We will get the: picture,price, product link, and description.

Resources used to help in this process can be found in other notebooks.

#### Import your needed Libraries:

In [10]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import cv2
import numpy as np 
import pandas as pd
import re
from PIL import Image
import requests
from io import BytesIO
from scraping_functions import *
import numpy as np
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Create variables for each of the URL

We will do this for each of the pages we will be pulling from on _Hershel, O My Bag & Whimsy and Row_.

In [ ]:
hershel_backpacks = 'https://herschel.com/shop/womens/backpacks'
hershel_fp = 'https://herschel.com/shop/womens/hip-packs'
hershel_totes = 'https://herschel.com/shop/womens/totes'
hershel_duffles = 'https://herschel.com/shop/womens/duffles'

O_my_bag = 'https://www.omybag.nl/product-category/women/women-bags/'

#only the first 13-ish bags
WR_bags = 'https://whimsyandrow.com/collections/bags-clutches-pouches?sort_by=title-ascending'

### O My Bag:

Now that we have our page variables set up, we can scrape the bags. After inspecting on the website it appears we can just use beautiful soup for this task.

In [ ]:
omb_page = requests.get(O_my_bag)
omb_soup = BeautifulSoup(omb_page.content, 'lxml')
divs_imgs = omb_soup.findAll('div', {'class':'content-product'})

In [ ]:
omb_links = omb_soup.findAll('a', {'class':'woocommerce-LoopProduct-link'})
omb_href = [x['href'] for x in omb_links]

Now let's grab the images:

In [ ]:
omb_images_tags = omb_soup.findAll('img', {'class':'attachment-woocommerce_thumbnail'})
omb_images = [x['src'] for x in omb_images_tags]

Now, let's get into the details like price and name:

In [ ]:
omb_prod_details = omb_soup.findAll('div', {'class':'product-details'})

In [ ]:
omb_names = [x.find('div', {'class': 'product-title'}).text.strip() for x in omb_prod_details]
omb_prices = [x.find('span', {'class': 'price'}).text.strip() for x in omb_prod_details]

### Create a DataFrame and use our function to download pictures :

Below we will use the lists created above and make a dataframe. Then we will define a function to download the images into our directory, and save the image file name in a new column of the DataFrame.

In [ ]:
table = [omb_names,omb_prices,omb_href,omb_images]
OMB_df = pd.DataFrame(table).transpose()

In [ ]:
OMB_df.rename( columns={0:'product_name', 1: 'product_price',2:'URL',3:'img_url'},inplace=True)

In [ ]:
OMB_df.head(3)

#### Save all images using an apply

In [ ]:
c = 0
OMB_df['img'] = OMB_df.img_url.apply(lambda x: save_src_image_apply('../../data/','omybag_',x))

In [ ]:
OMB_df.to_csv('../../omb_df.csv')

### Whimsy and Row:

In [ ]:
wr_page = requests.get(WR_bags)
wr_soup = BeautifulSoup(wr_page.content, 'lxml')
divs_wr = wr_soup.findAll('div', {'class':'product-frame'})

In [ ]:
wr_url = ['https://whimsyandrow.com' + x.find('a')['href'] for x in divs_wr[:-3]]
wr_names = [x.find('a')['title'] for x in divs_wr[:-3]]
wr_images_url = ['https:'+x.find('img')['src'] for x in divs_wr[:-3]] 

Now, let's get the prices which are in a different div:

In [ ]:
divs_wr_2 = wr_soup.findAll('div', {'class':'product-title'})
prices = [x.find('h6').text for x in divs_wr_2[:-3]]

### Create this whimsy and Row DF and save the images:

In [ ]:
table = [wr_names,prices,wr_url,wr_images_url]
wr_df = pd.DataFrame(table).transpose()
wr_df.rename(columns={0:'product_name', 1: 'product_price',2:'URL',3:'img_url'},inplace=True)

#### Save all images using an apply:

In [ ]:
c = 0
wr_df['img'] = wr_df.img_url.apply(lambda x: save_src_image_apply('../../data/','whimsy_row_',x))

In [ ]:
wr_df.to_csv('../../whimsy_row_df.csv')

### _Hershel_:

In [ ]:
h_page_1 = requests.get(hershel_backpacks)
h_soup_1 = BeautifulSoup(h_page_1.content, 'lxml')
divs_h = h_soup_1.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [ ]:
#three of the bookbags do not have multiple colors
colors_carousel = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]

h_bb_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel]

In [ ]:
h_bookbag_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_bb_imgs]
h_bookbag_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_bb_imgs]

In [ ]:
names = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]

In [ ]:
prices = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url = [x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

In [ ]:
names_adj = []
for i in range(len(names)):
    n = len(h_bb_imgs[i])
    names_adj.extend([names[i]]*n)

In [ ]:
prices_adj =  []
for i in range(len(prices)):
    n = len(h_bb_imgs[i])
    prices_adj.extend([prices[i]]*n)

In [ ]:
prod_h_url_adj =  []
for i in range(len(prod_h_url)):
    n = len(h_bb_imgs[i])
    prod_h_url_adj.extend([prod_h_url[i]]*n)

In [ ]:
len(h_bookbag_img),len(names)

In [ ]:
len(names_adj),len(prices_adj),len(prod_h_url_adj)

Let's flatten the lists of colors and images:

In [ ]:
h_bookbag_colors_fl = [color for sublist in h_bookbag_colors for color in sublist]
h_bookbag_img_fl = [img for sublist in h_bookbag_img for img in sublist]
# len(h_bookbag_colors_fl),len(h_bookbag_img_fl)

Make the DataFrame:

In [ ]:
table = [names_adj,prices_adj,h_bookbag_colors_fl,prod_h_url_adj,h_bookbag_img_fl]
herschel_df = pd.DataFrame(table).transpose()
herschel_df.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

#### Save all images using an apply:

In [ ]:
c = 0
herschel_df['img'] = herschel_df.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel',x))

In [ ]:
herschel_df.to_csv('../../herschel_df.csv')

### Herschel Fannie Packs:

In [ ]:
h_page_1 = requests.get(hershel_fp)
h_soup_1 = BeautifulSoup(h_page_1.content, 'lxml')
divs_h = h_soup_1.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [ ]:
colors_carousel_fp = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]
h_fp_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel_fp]

In [ ]:
h_fannie_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_fp_imgs]
h_fannie_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_fp_imgs]
names_fp = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]
prices_fp = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url_fp = ['https://herschel.com'+x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

Extend prices, names, and product url lists to ensure they have the same lengths as all of the colors.

In [ ]:
names_adj_fp = []
for i in range(len(names_fp)):
    n = len(h_bb_imgs_fp[i])
    names_adj_fp.extend([names_fp[i]]*n)
    
prices_adj_fp =  []
for i in range(len(prices_fp)):
    n = len(h_fp_imgs[i])
    prices_adj_fp.extend([prices_fp[i]]*n)

prod_h_url_adj_fp =  []
for i in range(len(prod_h_url_fp)):
    n = len(h_fp_imgs[i])
    prod_h_url_adj_fp.extend([prod_h_url_fp[i]]*n)

Flatten the images and colors lists:

In [ ]:
h_fp_colors_fl = [color for sublist in h_fannie_colors for color in sublist]
h_fp_img_fl = [img for sublist in h_fannie_img for img in sublist]
len(h_fp_colors_fl),len(h_fp_img_fl),len(prod_h_url_adj_fp)

Create the DataFrame of the Herschel Fannie Packs:

In [ ]:
table = [names_adj_fp,prices_adj_fp,h_fp_colors_fl,prod_h_url_adj_fp,h_fp_img_fl]
herschel_df_fp = pd.DataFrame(table).transpose()
herschel_df_fp.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

In [ ]:
herschel_df_fp.head()

#### Save all images using an apply:

In [ ]:
c = 0
herschel_df_fp['img'] = herschel_df_fp.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel_fp',x))

In [ ]:
herschel_df_fp.to_csv('../../herschel_df_fp.csv')

### Herschel Totes:

In [ ]:
h_page_1 = requests.get(hershel_totes)
h_soup_1 = BeautifulSoup(h_page_1.content, 'lxml')
divs_h = h_soup_1.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [ ]:
colors_carousel_t = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]
h_t_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel_t]

In [ ]:
h_tote_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_t_imgs]
h_tote_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_t_imgs]
names_t = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]
prices_t = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url_t = ['https://herschel.com'+x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

Extend prices, names, and product url lists to ensure they have the same lengths as all of the colors.

In [ ]:
names_adj_t = []
for i in range(len(names_t)):
    n = len(h_t_imgs[i])
    names_adj_t.extend([names_t[i]]*n)
    
prices_adj_t =  []
for i in range(len(prices_t)):
    n = len(h_t_imgs[i])
    prices_adj_t.extend([prices_t[i]]*n)
1
prod_h_url_adj_t =  []
for i in range(len(prod_h_url_t)):
    n = len(h_t_imgs[i])
    prod_h_url_adj_t.extend([prod_h_url_t[i]]*n)

Flatten the images and colors lists:

In [ ]:
h_t_colors_fl = [color for sublist in h_tote_colors for color in sublist]
h_t_img_fl = [img for sublist in h_tote_img for img in sublist]
len(h_t_colors_fl),len(h_t_img_fl),len(prod_h_url_adj_t)

Create the DataFrame of the Herschel Fannie Packs:

In [ ]:
table = [names_adj_t,prices_adj_t,h_t_colors_fl,prod_h_url_adj_t,h_t_img_fl]
herschel_df_t = pd.DataFrame(table).transpose()
herschel_df_t.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

In [ ]:
herschel_df_t.head()

#### Save all images using an apply:

In [ ]:
c = 0
herschel_df_t['img'] = herschel_df_t.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel_t',x))

In [ ]:
herschel_df_t.to_csv('../../herschel_df_t.csv')

### Herschel Weekenders:


In [ ]:
h_page_2 = requests.get(hershel_duffles)
h_soup_2 = BeautifulSoup(h_page_2.content, 'lxml')
divs_h = h_soup_2.findAll('div', {'class':'col-lg-4 col-md-6 col-xs-12 m-b-2'})

In [ ]:
colors_carousel_w = [x.findAll('div',{'class':'carousel'}) if x.findAll('div',{'class':'carousel'}) 
                   else x.find('div',{'class':'product-image'}) for x in divs_h]
h_w_imgs = [i[0].findAll('img',{'class':'swatch'}) if str(type(i)) == "<class 'bs4.element.ResultSet'>" 
             else [i.find('img')] for i in colors_carousel_w]

In [ ]:
h_wkndr_colors = [[i['data-color'] if 'data-color' in str(i) else 'NA' for i in x] for x in h_w_imgs]
h_wkndr_img = [['https://herschel.com'+i['data-product-image'] if 'data-product-image' in 
                  str(i) else 'https://herschel.com'+i['data-src'] for i in x] for x in h_w_imgs]
names_w = [x.find('div',{'class':'col-xs-6 title-line'}).text for x in divs_h]
prices_w = [x.find('span',{'class':'hsco-set-currency'}).text for x in divs_h]
prod_h_url_w = ['https://herschel.com'+x.find('a',{'class':'js-product-grid-link'})['href'] for x in divs_h]

Extend prices, names, and product url lists to ensure they have the same lengths as all of the colors.

In [ ]:
names_adj_w = []
for i in range(len(names_w)):
    n = len(h_w_imgs[i])
    names_adj_w.extend([names_w[i]]*n)
    
prices_adj_w =  []
for i in range(len(prices_w)):
    n = len(h_w_imgs[i])
    prices_adj_w.extend([prices_w[i]]*n)
1
prod_h_url_adj_w =  []
for i in range(len(prod_h_url_w)):
    n = len(h_w_imgs[i])
    prod_h_url_adj_w.extend([prod_h_url_w[i]]*n)

Flatten the images and colors lists:

In [ ]:
h_w_colors_fl = [color for sublist in h_wkndr_colors for color in sublist]
h_w_img_fl = [img for sublist in h_wkndr_img for img in sublist]
len(h_w_colors_fl),len(h_w_img_fl),len(prod_h_url_adj_w)

Create the DataFrame of the Herschel Fannie Packs:

In [ ]:
table = [names_adj_w,prices_adj_w,h_w_colors_fl,prod_h_url_adj_w,h_w_img_fl]
herschel_df_w = pd.DataFrame(table).transpose()
herschel_df_w.rename(columns={0:'product_name', 1: 'product_price',2:'color',3:'URL',4:'img_url'},inplace=True)

In [ ]:
herschel_df_w.head()

#### Save all images using an apply:

In [ ]:
c = 0
herschel_df_w['img'] = herschel_df_w.img_url.apply(lambda x: save_src_image_apply('../../data/','herschel_w',x))

In [ ]:
herschel_df_w.to_csv('../../herschel_df_w.csv')

### Nordstrom Circle Bags:

Import needed libraries:

In [5]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

Create the webdriver object and get the base url:

In [ ]:
base_url = 'https://shop.nordstrom.com/'
#create the chrome driver
start_time = time.time()
driver = webdriver.Chrome('/Users/elenasm7/.wdm/chromedriver/2.46/mac64/chromedriver') 
driver.get(base_url)

Navigate to the search bar, type in circle bags, and press return:

In [ ]:
#now, search for the item we need
driver.find_element_by_class_name('ZlVcfv').click()
time.sleep(1)
real_search = driver.find_element_by_class_name('ZaYgXf')
real_search.send_keys('circle bags')
time.sleep(1)
real_search.submit()
time.sleep(1)

Create a list of all of the bags:

In [ ]:
bags_2 = driver.find_elements_by_class_name('Z1AvvAr')

Using a list comprehension grab all of the bag's links:

In [ ]:
img_urls = [img_urls.append(i.get_attribute("href")) for i in bags_2]    

#now, search for the item we need
driver.find_element_by_class_name('ZlVcfv').click()
time.sleep(1)
real_search = driver.find_element_by_class_name('ZaYgXf')
real_search.send_keys('circle bags')
time.sleep(1)
real_search.submit()
time.sleep(1)

bags_2 = driver.find_elements_by_class_name('Z1AvvAr')
img_urls = []
for i in bags_2:
    img_urls.append(i.get_attribute("href"))

Create a function to get all of the details we need from each bag:

In [4]:
def get_attributes(image_l,price_l,name_l,color_l,url_l,brand_l,li):
    slp = np.random.randint(1,10,size=4)
    image_l.append(driver.find_element_by_class_name(li[0]).get_attribute('src'))
    price_l.append(driver.find_element_by_class_name(li[1]).text)
    time.sleep(slp[0])
    name_l.append(driver.find_element_by_class_name(li[2]).text)
    #      _2nXdo0 FCcEY _28PBKh ODo94
    time.sleep(slp[1])
    brand_l.append(driver.find_element_by_class_name(li[3]).text)
    time.sleep(slp[2])
    color_l.append(driver.find_element_by_class_name(li[4]).get_attribute('aria-label'))
#     _2ndY0C _2dAOFI
    time.sleep(slp[3])
    color_options = driver.find_elements_by_class_name(li[5])
    url_l.append(driver.current_url)

Now create a for loop and empty lists to save all of the information we need:

In [ ]:
class_names_1 = ['_2g1r2h','Z1WEo3w','_6VXnG',
                 'LGMSA','_1QT7uJ','Z1MJbnq']
price_li = []
name_li = []
color = []
brand = []
url_li = []
img_li = []

for bag in img_urls:
    driver.get(bag)
    color_options = driver.find_elements_by_class_name('Z1MJbnq')
    if len(color_options) > 1:
        color_options[0].click()
        time.sleep(1)
        get_attributes(img_li,price_li,name_li,
                       color,url_li,brand,class_names_1)
        for i in color_options[1:]:
            time.sleep(1)
            i.click()
            get_attributes(img_li,price_li,name_li,
                           color,url_li,brand,class_names_1)
    else:
        time.sleep(1)
        get_attributes(img_li,price_li,name_li,
                       color,url_li,brand,class_names_1)
        

Follow the steps from before. Create a new data frame, and use the apply statement to download and save all of the bag images.

### Nordstrom fannie packs:
following the same steps as before to grab all the fannie packs from Nordstom.

In the next cell creating our base_url variable, and startng a new chromedriver session. Then navigating to the Nordstrom page:

In [105]:
base_url = 'https://shop.nordstrom.com/'
#create the chrome driver
start_time = time.time()
driver = webdriver.Chrome('/Users/elenasm7/.wdm/chromedriver/76.0.3809.126/mac64/chromedriver') 
driver.get(base_url)
# Z1craAD

below, we search for fanny packs:

In [89]:
#now, search for the item we need
driver.find_element_by_class_name('_254wwN').click()
time.sleep(1)
real_search = driver.find_element_by_class_name('_27R4o5')
# ZaYgXf
real_search.send_keys('fanny packs')
time.sleep(1)
real_search.submit()
time.sleep(1)

'https://shop.nordstrom.com/s/madewell-the-simple-pouch-belt-bag/5428490?origin=keywordsearch-personalizedsort&breadcrumb=Home%2FAll%20Results&color=english%20saddle'

Next, we go through each page of bags, grab all of their urls, then grab the img urls:

In [100]:
pages = ['2','3','4','5','6','7']
bag_urls = []
current_fp = driver.find_elements_by_class_name('Z1AvvAr')
bag_urls.extend([i.get_attribute("href") for i in current_fp])

for i in pages:
    time.sleep(2)
    driver.find_element_by_link_text(i).click()
    time.sleep(1)
    current_fp = driver.find_elements_by_class_name('Z1AvvAr')
    bag_urls.extend([i.get_attribute("href") for i in current_fp])
    

made updates to the get_attributes function above and added parameter li. When using the function you now pass it a list of class names. Below we use the function the same as before by looping through each bag item and getting the features we need.

In [102]:
class_names = ['Z1rpNm4','Z1WEo3w','_2nXdo0','bpCeO','JniDt']

In [28]:
def get_attributes(image_l,price_l,name_l,color_l,url_l,brand_l,li,c):
    slp = np.random.randint(1,5,size=4)
    image_l.append(driver.find_element_by_class_name(li[0]).get_attribute('src'))
    price_l.append(driver.find_element_by_class_name(li[1]).text)
    time.sleep(slp[0])
    name_l.append(driver.find_element_by_class_name(li[2]).text)
    #      _2nXdo0 FCcEY _28PBKh ODo94
    time.sleep(slp[1])
    brand_l.append(driver.find_element_by_class_name(li[3]).text)
    time.sleep(slp[2])
    color_l.append(driver.find_elements_by_class_name(li[4])[c].get_attribute('aria-label'))
#     _2ndY0C _2dAOFI
    time.sleep(slp[3])
    url_l.append(driver.current_url)

In [107]:
price_li = []
name_li = []
color = []
brand = []
url_li = []
img_li = []

for bag in bag_urls:
    driver.get(bag)
    color_options = driver.find_elements_by_class_name('Z1MJbnq')
    counter = 0
    if len(color_options) > 1:
        color_options[0].click()
        time.sleep(1.5)
        get_attributes(img_li,price_li,name_li,color,
                       url_li,brand,class_names,counter)
        for i in color_options[1:]:
            counter += 1
            time.sleep(2)
            i.click()
            get_attributes(img_li,price_li,name_li,color,
                           url_li,brand,class_names,counter)
    else:
        time.sleep(1)
        get_attributes(img_li,price_li,name_li,color,
                       url_li,brand,class_names,counter)

In [108]:
nord_fp = pd.DataFrame(list(zip(price_li, name_li, color, brand, url_li, img_li)),
                       columns=['price','name','color','brand','url','img_url'])

In [109]:
nord_fp.shape

(626, 6)

Pickle the DataFrame:

In [110]:
nord_fp.to_pickle('nord_fp_df.pkl')

Update brand names to make file naming more simple. We followed this by creating a new column of a shortened brand name which we will use:

In [114]:
nord_fp.brand = nord_fp.brand.apply(lambda x: x.lower().replace('.',''))

In [125]:
nord_fp['new_name'] = nord_fp.brand.apply(lambda x: x.split(' ')[0])

Use the function defined in the scraping script. We use it as an apply on each column (axis=1). We pass it the file location, the column of the brand name, and the image url:

In [ ]:
c = 0
nord_fp['filename'] = nord_fp.apply(lambda x: save_src_image_apply('../../../../img/fanny_packs/','nord-'+x[6],x[5]),axis=1)


Update the DataFrame and get rid of the updated brand name:

In [133]:
nord_fp_new = nord_fp[['price', 'name', 'color', 'brand', 'url', 'img_url',
       'filename']]

Pickle the final DataFrame:

In [135]:
nord_fp_new.to_pickle('nord_fp_df.pkl')

In [137]:
# nord_fp_new

print number of files in each subdirectory, using the fuction - *subdir_len* - defined in scraping_scripts:

In [144]:
import os
# path = '../../../../img/'
        
subdir_len('../../../../img/')

278 buckets
1011 fanny_packs
163 circles
1749 shoulder_bags
1550 totes
281 weekenders
345 backpacks
486 clutches


By this we can tell we need more buckets, circles, weekenders, backpacks, and clutches. Since we already scraped circles from Nord%&#\*m, we will be doing the other ones here.

### Buckets:

In [7]:
base_url = 'https://shop.nordstrom.com/'
#create the chrome driver
start_time = time.time()
driver = webdriver.Chrome('/Users/elenasm7/.wdm/chromedriver/76.0.3809.126/mac64/chromedriver') 
driver.get(base_url)

In [8]:
driver.find_element_by_class_name('_2Kwni').click()
time.sleep(1)
real_search = driver.find_element_by_class_name('_2341R')
# ZaYgXf
real_search.send_keys('bucket bags')
time.sleep(1)
real_search.submit()
time.sleep(1)

In [12]:
pages = ['2','3']
bb_urls = []
current_bb = driver.find_elements_by_class_name('_1av3_')
bb_urls.extend([i.get_attribute("href") for i in current_bb])

for i in pages:
    slp = np.random.randint(1,10,size=1)
    time.sleep(slp[0])
    driver.find_element_by_link_text(i).click()
    time.sleep(1)
    current_bb = driver.find_elements_by_class_name('_1av3_')
    bb_urls.extend([i.get_attribute("href") for i in current_bb])

In [26]:
class_names_bb = ['_1xc1x','_3p7kp','UlMCr','_36bPE','_2fvOm']

In [52]:
# price_li_bb = []
# name_li_bb = []
# color_bb = []
# brand_bb = []
# url_li_bb = []
# img_li_bb = []

for bag in bb_urls[26:]:
    driver.get(bag)
    color_options = driver.find_elements_by_class_name('_3lh0_')
    counter = 0
    if len(color_options) > 1:
        color_options[0].click()
        time.sleep(1.5)
        get_attributes(img_li_bb,price_li_bb,name_li_bb,color_bb,
                       url_li_bb,brand_bb,class_names_bb,counter)
        for i in color_options[1:]:
            counter += 1
            time.sleep(1)
            i.click()
            get_attributes(img_li_bb,price_li_bb,name_li_bb,color_bb,
                           url_li_bb,brand_bb,class_names_bb,counter)
    else:
        time.sleep(1)
        get_attributes(img_li_bb,price_li_bb,name_li_bb,color_bb,
                       url_li_bb,brand_bb,class_names_bb,counter)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"._3p7kp"}
  (Session info: chrome=76.0.3809.132)


In [45]:
price_li_bb = price_li_bb[:48]
name_li_bb = name_li_bb[:48]
color_bb = color_bb[:48]
brand_bb = brand_bb[:48]
url_li_bb = url_li_bb[:48]
img_li_bb = img_li_bb[:48]

In [40]:
nord_bb = pd.DataFrame(list(zip(price_li_bb, name_li_bb, color_bb, brand_bb, url_li_bb, img_li_bb)),
                       columns=['price','name','color','brand','url','img_url'])

In [53]:
nord_bb_2 = pd.DataFrame(list(zip(price_li_bb, name_li_bb, color_bb, brand_bb, url_li_bb, img_li_bb)),
                       columns=['price','name','color','brand','url','img_url'])

In [55]:
nord_bb_2.to_pickle('nord_bb_2_df.pkl')
nord_bb.to_pickle('nord_bb_df.pkl')

In [42]:
nord_bb.iloc[:48,:]

,price,name,color,brand,url,img_url
0,$169.00,Delilah Leather Bucket Bag,Black,NORDSTROM,https://shop.nordstrom.com/s/nordstrom-delilah...,https://n.nordstrommedia.com/id/sr3/52660e38-4...
1,$169.00,Delilah Leather Bucket Bag,Brown Aztec,NORDSTROM,https://shop.nordstrom.com/s/nordstrom-delilah...,https://n.nordstrommedia.com/id/sr3/ae92c04f-a...
2,$169.00,Delilah Leather Bucket Bag,Yellow Inca,NORDSTROM,https://shop.nordstrom.com/s/nordstrom-delilah...,https://n.nordstrommedia.com/id/sr3/d02a4b1f-9...
3,$980.00,Mini GG Marmont 2.0 Quilted Leather Bucket Bag,Hibiscus Red,GUCCI,https://shop.nordstrom.com/s/gucci-mini-gg-mar...,https://n.nordstrommedia.com/id/sr3/4b4454e4-9...
4,$980.00,Mini GG Marmont 2.0 Quilted Leather Bucket Bag,Mystic White,GUCCI,https://shop.nordstrom.com/s/gucci-mini-gg-mar...,https://n.nordstrommedia.com/id/sr3/6629e758-9...
5,$980.00,Mini GG Marmont 2.0 Quilted Leather Bucket Bag,Nero,GUCCI,https://shop.nordstrom.com/s/gucci-mini-gg-mar...,https://n.nordstrommedia.com/id/sr3/1d6ff9b8-b...
6,$980.00,Mini GG Marmont 2.0 Quilted Leather Bucket Bag,Porcelain Rose,GUCCI,https://shop.nordstrom.com/s/gucci-mini-gg-mar...,https://n.nordstrommedia.com/id/sr3/2c0f1a22-c...
7,$178.80,suzy small leather bucket bag,Black,KATE SPADE NEW YORK,https://shop.nordstrom.com/s/kate-spade-new-yo...,https://n.nordstrommedia.com/id/sr3/b7e531ba-c...
8,$178.80,suzy small leather bucket bag,Blazer Blue,KATE SPADE NEW YORK,https://shop.nordstrom.com/s/kate-spade-new-yo...,https://n.nordstrommedia.com/id/sr3/cf6261da-f...
9,$178.80,suzy small leather bucket bag,Cinnamon Spice,KATE SPADE NEW YORK,https://shop.nordstrom.com/s/kate-spade-new-yo...,https://n.nordstrommedia.com/id/sr3/1a13979d-c...


In [44]:
name_li_bb[:48]

['Delilah Leather Bucket Bag',
 'Delilah Leather Bucket Bag',
 'Delilah Leather Bucket Bag',
 'Mini GG Marmont 2.0 Quilted Leather Bucket Bag',
 'Mini GG Marmont 2.0 Quilted Leather Bucket Bag',
 'Mini GG Marmont 2.0 Quilted Leather Bucket Bag',
 'Mini GG Marmont 2.0 Quilted Leather Bucket Bag',
 'suzy small leather bucket bag',
 'suzy small leather bucket bag',
 'suzy small leather bucket bag',
 'Campbell Leather & Suede Bucket Bag',
 'Campbell Leather & Suede Bucket Bag',
 'Mini Miller Leather Bucket Bag',
 'Mini Miller Leather Bucket Bag',
 'Mini Miller Leather Bucket Bag',
 'Fable Floral Straw Bucket Bag',
 'Small Robinson Croc Embossed Leather Bucket Bag',
 'Austin Crossbody Bag',
 'Austin Crossbody Bag',
 'Austin Crossbody Bag',
 'Two Lovers Vegan Leather Bucket Bag',
 'Mini Mini Leather Bucket Bag',
 'Mini Mini Leather Bucket Bag',
 'Mini Mini Leather Bucket Bag',
 'Sydney Patchwork Eel Leather Bucket Bag',
 'Mini Mini Leather Bucket Bag',
 'Mini Mini Leather Bucket Bag',
 'Smal

In [50]:
bb_urls[26]

'https://shop.nordstrom.com/s/yuzefi-delia-bucket-bag/4953875?origin=keywordsearch-personalizedsort&breadcrumb=Home%2FAll%20Results&color=bruciato%2F%20kaky'